# AI Assistant with Face Recognition

This Jupyter Notebook contains an AI assistant that uses OpenCV, MediaPipe, and DeepFace to detect faces, recognize users, and assist them via voice commands.

Each function is placed in a separate cell for clarity.


In [1]:
import pyttsx3

# Initialize voice engine
engine = pyttsx3.init()

def speak(text):
    """Converts text to speech."""
    engine.say(text)
    engine.runAndWait()


In [2]:
import speech_recognition as sr

recognizer = sr.Recognizer()

def get_voice_command():
    """Captures and recognizes a voice command from the user."""
    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source)
        speak("Listening for your command...")
        try:
            audio = recognizer.listen(source, timeout=5)
            command = recognizer.recognize_google(audio).lower()
            return command
        except sr.UnknownValueError:
            return ""
        except sr.RequestError:
            return ""


In [3]:
def assist_user():    
    # Ask the user how they need help
    prompt = "How can I help you? Please choose an option: \n1. Meet Dr. Hashim \n2. Meet Mr. Misbah \n3. Meet Mr. Taha"
    print(prompt)
    engine.say("How can I help you? Please choose an option.")
    engine.runAndWait()
    engine.say("Press 1 to meet Doctor Hashim, 2 for Mr. Misbah, or 3 for Mr. Taha.")
    engine.runAndWait()
    
    # Get user input
    choice = input("Enter your choice (1/2/3): ").strip()
    
    # Process the choice
    if choice == "1":
        response = "You have chosen to meet Dr. Hashim."
    elif choice == "2":
        response = "You have chosen to meet Mr. Misbah."
    elif choice == "3":
        response = "You have chosen to meet Mr. Taha."
    else:
        response = "Invalid choice. Please try again."
    
    # Speak the response
  
    engine.say(response)
    engine.runAndWait()
    print(response)
    return response

In [4]:
import cv2
import os
import json

USER_DATA_FILE = "user_data.json"

if os.path.exists(USER_DATA_FILE):
    with open(USER_DATA_FILE, "r") as f:
        user_data = json.load(f)
else:
    user_data = {}

def capture_new_face(face_image):
    """Captures and stores a new user's face with their name."""
    speak("Who are you? Please enter your name.")
    name = input("Please enter your name: ")
    save_dir = r"F:\Buffer\Applied data science and AI specialization\NCL open cv tasks\image database"
    os.makedirs(save_dir, exist_ok=True)  # Ensure directory exists
    save_path = os.path.join(save_dir, f"{name}.jpg")

    cv2.imwrite(save_path, face_image)
    user_data[name] = {"greet_count": 0}
    with open(USER_DATA_FILE, "w") as f:
        json.dump(user_data, f)
    speak(f"Image saved. Welcome, {name}!")


In [5]:
from deepface import DeepFace

def detect_emotion(face_image):
    """Analyzes the detected face for emotion."""
    try:
        result = DeepFace.analyze(face_image, actions=["emotion"], enforce_detection=False)
        if result:
            return result[0]['dominant_emotion']
    except:
        return "neutral"
    return "neutral"


In [6]:
def detect_objects(frame):
    """Detects objects in the given frame using YOLO."""
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    
    objects_detected = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = scores.argmax()
            confidence = scores[class_id]
            if confidence > 0.5:
                objects_detected.append(class_id)
    
    return objects_detected


In [7]:
import os
import json
import threading
from deepface import DeepFace

def find_face():
    try:
        results = DeepFace.find(
            img_path='temp_face.jpg',
            db_path=r"F:\Buffer\Applied data science and AI specialization\NCL open cv tasks\image database",
            model_name="Facenet",
            enforce_detection=False
        )

        if results and len(results[0]) > 0:
            name = os.path.basename(results[0]['identity'][0]).split('.')[0]

            if name in user_data:
                user_data[name]['greet_count'] += 1
            else:
                user_data[name] = {"greet_count": 1}

            with open(USER_DATA_FILE, "w") as f:
                json.dump(user_data, f)

            speak(f"Welcome back, {name}!")
            emotion = detect_emotion(face_image)

            if emotion == "sad":
                speak("You seem a bit down today. Can I do something for you?")
            elif emotion == "happy":
                speak("You look happy! Glad to see you smiling!")

            threading.Thread(target=assist_user).start()
        else:
            speak("Face not recognized. Who are you?")
            threading.Thread(target=capture_new_face, args=(face_image,)).start()

    except Exception as e:
        print("Error:", e)
        speak("An error occurred while recognizing your face.")


In [8]:
import mediapipe as mp
import time
import threading

# Initialize Face Detection
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

cap = cv2.VideoCapture(0)

start_time = None
face_detected = False
greeted = False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    small_frame = cv2.resize(frame, (640, 480))
    rgb_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
    results = face_detection.process(rgb_frame)
    
    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            h, w, _ = small_frame.shape
            x, y, w, h = int(bboxC.xmin * w), int(bboxC.ymin * h), int(bboxC.width * w), int(bboxC.height * h)
            cv2.rectangle(small_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        if not face_detected:
            start_time = time.time()
            face_detected = True
        
        time_passed = time.time() - start_time
        if time_passed > 3.0 and not greeted:
            greeted = True
            face_image = small_frame[y:y+h, x:x+w]
            cv2.imwrite("temp_face.jpg",frame)  # Always overwrite this file
            threading.Thread(target=find_face).start()
    else:
        start_time = None
        face_detected = False
        greeted = False
    
    cv2.imshow("AI Assistant", small_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


25-03-21 11:43:34 - Searching temp_face.jpg in 4 length datastore
25-03-21 11:43:59 - find function duration 24.906829595565796 seconds
How can I help you? Please choose an option: 
1. Meet Dr. Hashim 
2. Meet Mr. Misbah 
3. Meet Mr. Taha


Enter your choice (1/2/3):  3


You have chosen to meet Mr. Taha.
25-03-21 11:44:34 - Searching temp_face.jpg in 4 length datastore
25-03-21 11:44:34 - find function duration 0.7455530166625977 seconds
How can I help you? Please choose an option: 
1. Meet Dr. Hashim 
2. Meet Mr. Misbah 
3. Meet Mr. Taha
